In [1]:
# import json
import os
import fitz  # PyMuPDF
import re

# Get Document

# Clean Document and get Structure

In [75]:
source = 'C:\\Users\\gciprianherrera\\Desktop\\LLM\MVP_Chatbot\\PDF\\DEG\\ELH-2020-033633.pdf'
# source = 'C:\\Users\\gciprianherrera\\Desktop\\LLM\MVP_Chatbot\\PDF\\DEG\\ELH-2021-001129.pdf'

<>:1: SyntaxWarning: invalid escape sequence '\M'
<>:1: SyntaxWarning: invalid escape sequence '\M'
C:\Users\gciprianherrera\AppData\Local\Temp\ipykernel_15268\889672308.py:1: SyntaxWarning: invalid escape sequence '\M'
  source = 'C:\\Users\\gciprianherrera\\Desktop\\LLM\MVP_Chatbot\\PDF\\DEG\\ELH-2020-033633.pdf'


In [76]:
def get_pdf_file(source: str):
    doc = fitz.open(source)
    return doc

In [77]:
def find_title(lines, start_index, end_index):
    title = "Title not found"
    if lines[end_index].startswith(("Page", "PAGE", "P1"))==False:
        title = lines[start_index+1:end_index-1]
        title = " ".join(title)
    else:
        title = lines[start_index+1:end_index]
        title = " ".join(title)
    return title

In [78]:
def merge_lines_content(lines):
    """Merge lines where a line does not end with a period and the next starts with a lowercase letter."""
    merged_lines = []
    i = 0
    while i < len(lines):
        if i < len(lines) - 1 and not lines[i].endswith('.') and lines[i+1][0].islower():
            merged_lines.append(lines[i] + ' ' + lines[i+1])
            i += 2
        else:
            merged_lines.append(lines[i])
            i += 1
    return merged_lines

In [82]:
def process_document_information(source):
    
    doc = get_pdf_file(source)
    page = doc[0].get_text()
    lines = page.split('\n')
    lines = [line for line in lines if line.strip()]
    
    
    # Find the index of the line that contains the word "TABLES DES MATIERES"
    toc_index = next((i for i, line in enumerate(lines) if "TABLES DES MATIERES" in line or "TABLE DES MATIERES" in line), None)
    # Find the index where the references start
    references_index = next((i for i, line in enumerate(lines) if re.match(r"ELH-\d{4}-\d{6}(?:_\d{1})?", line)), None)
    
    # Find the index of the line that contains the word "TABLES DES MATIERES"
    compte_rendu_index = next((i for i, line in enumerate(lines) if re.match(r"\bCOMPTE[-\s]RENDU\b", line)), None)
    
    info_content = lines[:toc_index] + lines[references_index:]
    # metadata['Title'] = find_title(info_content, compte_rendu_index, compte_rendu_index+3)
    title = find_title(info_content, compte_rendu_index, compte_rendu_index+3)
    
    for line in info_content:
        if re.match(r"(\d\.\d)", line):
            version = line
            break
        
    for line in info_content:
        if re.match(r"\b\d{2}/\d{2}/\d{4}\b", line):
            date = line
            break
    
    for line in info_content:
        if re.match(r"(Diffusion (?:Limitée|Normale) Orano)", line):
            confidentiality = line
            break
        
    perimeter = os.path.basename(os.path.dirname(source))
    
    investigation_number = os.path.basename(source).split(".")[0]
    
    metadata = {
        "Title": title,
        "Version": version,
        "Date": date,
        "Confidentiality": confidentiality,
        "Perimeter": perimeter,
        "Investigation Number": investigation_number
    }
    return metadata

In [ ]:
process_document_information(source)

In [ ]:
def process_document_body(soure):
    # Initialize indices
    doc = get_pdf_file(source)
    toc_index = None
    references_index = None
    page_content = []

    for page in doc:
        # Get text from the page
        text = page.get_text()
        
        # Split text into lines and remove empty lines
        lines = text.split('\n')
        lines = [line for line in lines if line.strip()]
        lines = [line for line in lines if '\uf0b7' not in line]
        
        # If the page is the first page
        if page.number == 0:
        
            # Find the index of the line that contains the word "TABLES DES MATIERES"
            toc_index = next((i for i, line in enumerate(lines) if "TABLES DES MATIERES" in line or "TABLE DES MATIERES" in line), None)

            # toc_index = next((i for i, line in enumerate(lines) if re.match(r"TABLE[S]? DE[S]? MATIERE[S]?", line, re.IGNORECASE)), None)
            # toc_index = next((i for i, line in enumerate(lines) if "TABLES DES MATIERES" in line), None)
            
            # Find the index where the references start
            references_index = next((i for i, line in enumerate(lines) if re.match(r"ELH-\d{4}-\d{6}(?:_\d{1})?", line)), None)
            
            # Assign default values if indices are None
            toc_index = toc_index if toc_index is not None else -1
            references_index = references_index if references_index is not None else len(lines)
        
        # For all pages, use the determined indices
        references_index = next((i for i, line in enumerate(lines) if re.match(r"ELH-\d{4}-\d{6}(?:_\d{1})?", line)), None)
        content = lines[toc_index:references_index]
        # page_content = add_hyperlink_to_figure_lines(content)
        
        # Merge lines content
        while True:
            new_content = merge_lines_content(content)
            if new_content == content:
                break
            content = new_content 
        
        page_content.extend(content)
    
    page_content = "\n".join(page_content)
    return page_content

process_document_body(source)

In [ ]:
def get_pdf_file(source: str):
    doc = fitz.open(source)
    return doc

def add_hyperlink_to_figure_lines(lines):
    """Add hyperlinks to lines containing the word 'Figure' followed by a number."""
    figure_pattern = re.compile(r"Figure \d+")
    linked_lines = []
    for line in lines:
        if figure_pattern.search(line):
            # Modify this line to include a hyperlink (example format)
            line = f'<a href="#figure"></a>{line}'
        linked_lines.append(line)
    return linked_lines

def merge_lines_content(lines):
    """Merge lines where a line does not end with a period and the next starts with a lowercase letter."""
    merged_lines = []
    i = 0
    while i < len(lines):
        if i < len(lines) - 1 and not lines[i].endswith('.') and lines[i+1][0].islower():
            merged_lines.append(lines[i] + ' ' + lines[i+1])
            i += 2
        else:
            merged_lines.append(lines[i])
            i += 1
    return merged_lines

def find_title(lines, start_index, end_index):
    title = "Title not found"
    if lines[end_index].startswith(("Page", "PAGE", "P1"))==False:
        title = lines[start_index+1:end_index-1]
        title = " ".join(title)
    else:
        title = lines[start_index+1:end_index]
        title = " ".join(title)
    return title

def process_document_body(soure):
    # Initialize indices
    doc = get_pdf_file(source)
    toc_index = None
    references_index = None
    full_content = []

    for page in doc:
        # Get text from the page
        text = page.get_text()
        
        # Split text into lines and remove empty lines
        lines = text.split('\n')
        lines = [line for line in lines if line.strip()]
        lines = [line for line in lines if '\uf0b7' not in line]
        
        # If the page is the first page
        if page.number == 0:
            # Find the index of the line that contains the word "TABLES DES MATIERES"
            toc_index = next((i for i, line in enumerate(lines) if re.match(r"TABLE[S]? DE[S]? MATIERES?", line)), None)
            
            # Find the index where the references start
            references_index = next((i for i, line in enumerate(lines) if re.match(r"ELH-\d{4}-\d{6}(?:_\d{1})?", line)), None)
            
            # Assign default values if indices are None
            toc_index = toc_index if toc_index is not None else -1
            references_index = references_index if references_index is not None else len(lines)
        
        # For all pages, use the determined indices
        references_index = next((i for i, line in enumerate(lines) if re.match(r"ELH-\d{4}-\d{6}(?:_\d{1})?", line)), None)
        page_content = lines[toc_index:references_index]
        page_content = add_hyperlink_to_figure_lines(page_content)
        
        # Merge lines content
        while True:
            new_page_content = merge_lines_content(page_content)
            if new_page_content == page_content:
                break
            page_content = new_page_content 
        
        full_content.extend(page_content)
    
    full_text = "\n".join(full_content)
    return full_text

def process_document_information(source):
    
    doc = get_pdf_file(source)
    page = doc[0].get_text()
    lines = page.split('\n')
    lines = [line for line in lines if line.strip()]
    
    
    # Find the index of the line that contains the word "TABLES DES MATIERES"
    toc_index = next((i for i, line in enumerate(lines) if "TABLES DES MATIERES" in line), None)

    # Find the index where the references start
    references_index = next((i for i, line in enumerate(lines) if re.match(r"ELH-\d{4}-\d{6}(?:_\d{1})?", line)), None)
    
    # Find the index of the line that contains the word "TABLES DES MATIERES"
    compte_rendu_index = next((i for i, line in enumerate(lines) if re.match(r"\bCOMPTE[-\s]RENDU\b", line)), None)
    
    info_content = lines[:toc_index] + lines[references_index:]
    # metadata['Title'] = find_title(info_content, compte_rendu_index, compte_rendu_index+3)
    title = find_title(info_content, compte_rendu_index, compte_rendu_index+3)
    
    for line in info_content:
        if re.match(r"(\d\.\d)", line):
            version = line
            break
        
    for line in info_content:
        if re.match(r"\b\d{2}/\d{2}/\d{4}\b", line):
            date = line
            break
    
    for line in info_content:
        if re.match(r"(Diffusion (?:Limitée|Normale) Orano)", line):
            confidentiality = line
            break
        
    perimeter = os.path.basename(os.path.dirname(source))
    
    investigation_number = os.path.basename(source).split(".")[0]
    
    metadata = {
        "Title": title,
        "Version": version,
        "Date": date,
        "Confidentiality": confidentiality,
        "Perimeter": perimeter,
        "Investigation Number": investigation_number
    }
    return metadata

# create a dictionnary with the metadata and the full text
def process_document(source):
    metadata = process_document_information(source)
    full_text = process_document_body(source)
    document = {
        "metadata": metadata,
        "full_text": full_text
    }
    return document

process_document(source)


In [ ]:
import fitz  # PyMuPDF
import re
import os

class PDFDocumentProcessor:
    def __init__(self, source: str):
        source = source
        doc = get_pdf_file()
        toc_index = None
        references_index = None
        page_content = []
    
    def get_pdf_file(self):
        doc = fitz.open(source)
        return doc
    
    def add_hyperlink_to_figure_lines(self, lines):
        """Add hyperlinks to lines containing the word 'Figure' followed by a number."""
        figure_pattern = re.compile(r"Figure \d+")
        linked_lines = []
        for line in lines:
            if figure_pattern.search(line):
                # Modify this line to include a hyperlink (example format)
                line = f'<a href="#figure"></a>{line}'
            linked_lines.append(line)
        return linked_lines

    def merge_lines_content(self, lines):
        """Merge lines where a line does not end with a period and the next starts with a lowercase letter."""
        merged_lines = []
        i = 0
        while i < len(lines):
            if i < len(lines) - 1 and not lines[i].endswith('.') and lines[i+1][0].islower():
                merged_lines.append(lines[i] + ' ' + lines[i+1])
                i += 2
            else:
                merged_lines.append(lines[i])
                i += 1
        return merged_lines
    
    def find_title(self, lines, start_index, end_index):
        title = "Title not found"
        if not lines[end_index].startswith(("Page", "PAGE", "P1")):
            title = lines[start_index+1:end_index-1]
            title = " ".join(title)
        else:
            title = lines[start_index+1:end_index]
            title = " ".join(title)
        return title
    
    def process_document_body(self):
        for page in doc:
            # Get text from the page
            text = page.get_text()
            
            # Split text into lines and remove empty lines
            lines = text.split('\n')
            lines = [line for line in lines if line.strip()]
            lines = [line for line in lines if '\uf0b7' not in line]
            
            # If the page is the first page
            if page.number == 0:
                # Find the index of the line that contains the word "TABLES DES MATIERES"
                toc_index = next((i for i, line in enumerate(lines) if re.match(r"TABLE[S]? DE[S]? MATIERES?", line)), None)
                
                # Find the index where the references start
                references_index = next((i for i, line in enumerate(lines) if re.match(r"ELH-\d{4}-\d{6}(?:_\d{1})?", line)), len(lines))
            
            # For all pages, use the determined indices
            references_index = next((i for i, line in enumerate(lines) if re.match(r"ELH-\d{4}-\d{6}(?:_\d{1})?", line)), len(lines))
            content = lines[toc_index:references_index]
            content = add_hyperlink_to_figure_lines(content)
            
            # Merge lines content
            while True:
                new_content = merge_lines_content(content)
                if new_content == content:
                    break
                content = new_content 
            
            page_content.extend(content)
        
        page_content = "\n".join(page_content)
        return page_content

    def process_document_information(self):
        page = doc[0].get_text()
        lines = page.split('\n')
        lines = [line for line in lines if line.strip()]
        
        # Find the index of the line that contains the word "TABLES DES MATIERES"
        toc_index = next((i for i, line in enumerate(lines) if re.match(r"TABLE[S]? DE[S]? MATIERES?", line)), None)

        # Find the index where the references start
        references_index = next((i for i, line in enumerate(lines) if re.match(r"ELH-\d{4}-\d{6}(?:_\d{1})?", line)), None)
        
        # Find the index of the line that contains the word "TABLES DES MATIERES"
        compte_rendu_index = next((i for i, line in enumerate(lines) if re.match(r"\bCOMPTE[-\s]RENDU\b", line)), None)
        
        info_content = lines[:toc_index] + lines[references_index:]
        title = find_title(info_content, compte_rendu_index, compte_rendu_index+3)
        
        version = None
        for line in info_content:
            if re.match(r"(\d\.\d)", line):
                version = line
                break
        
        date = None
        for line in info_content:
            if re.match(r"\b\d{2}/\d{2}/\d{4}\b", line):
                date = line
                break
        
        confidentiality = None
        for line in info_content:
            if re.match(r"(Diffusion (?:Limitée|Normale) Orano)", line):
                confidentiality = line
                break
        
        perimeter = os.path.basename(os.path.dirname(source))
        investigation_number = os.path.basename(source).split(".")[0]
        
        metadata = {
            "Title": title,
            "Version": version,
            "Date": date,
            "Confidentiality": confidentiality,
            "Perimeter": perimeter,
            "Investigation Number": investigation_number
        }
        return metadata

    def process_document(self):
        metadata = process_document_information()
        full_text = process_document_body()
        document = {
            "metadata": metadata,
            "full_text": full_text
        }
        return document

# Usage
source = "C:\\Users\\gciprianherrera\\Desktop\\LLM\\MVP_Chatbot\\PDF\\DEG\\ELH-2019-017580.pdf"
processor = PDFDocumentProcessor(source)
document = processor.process_document()
document

# Create dataset

In [1]:
import os
import pandas as pd
from Document import PDFDocumentProcessor

In [43]:

def extract_metadata_and_content(document):
        metadata = document.get('metadata', {})
        page_content = document.get('page_content', '')
        return metadata, page_content

data = []
start_directory = "C:\\Users\\gciprianherrera\\Desktop\\LLM\\MVP_Chatbot\\PDF"

for root, dirs, files in os.walk(start_directory):
    for file in files:
        if file.endswith('.pdf'):  # Adjust the file extension as needed
            file_path = os.path.join(root, file)
            processor = PDFDocumentProcessor(file_path)
            document = processor.process_document()
            metadata, page_content = extract_metadata_and_content(document)
            row = {
                'Title': metadata.get('Title'),
                'Version': metadata.get('Version'),
                'Date': metadata.get('Date'),
                'Confidentiality': metadata.get('Confidentiality'),
                'Perimeter': metadata.get('Perimeter'),
                'Investigation Number': metadata.get('Investigation Number'),
                'Page Content': page_content
            }
            data.append(row)
data = pd.DataFrame(data)

In [ ]:
from Dataset import DatasetBuilder

start_directory = "C:\\Users\\gciprianherrera\\Desktop\\LLM\\MVP_Chatbot\\PDF"
pdf_processor = DatasetBuilder(start_directory)
data = pdf_processor.process_directory()

# Chunking Data

In [5]:
import os
from semantic_router.encoders import OpenAIEncoder
from semantic_chunkers import StatisticalChunker
import pandas as pd

# Function to initialize the encoder
def initialize_encoder():
    # Set the OpenAI API key from environment variable or prompt the user
    os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
    # Initialize the encoder
    encoder = OpenAIEncoder(name="text-embedding-3-large")
    
    return encoder

# Function to create a chunker
def create_chunker(encoder):
    return StatisticalChunker(encoder=encoder)

# Function to process the dataset
def process_dataset(data):
    encoder = initialize_encoder()
    chunker = create_chunker(encoder)
    all_record_metadata = []

    for idx, document in data.iterrows():
        record_texts = chunker([document['Page Content']])
        processed_texts = []
        for chunk in record_texts[0]:
            text_chunk = chunk.splits
            text_chunk = " ".join(text_chunk)
            processed_texts.append(text_chunk)

        metadata = {
            'Title': document['Title'],
            'Version': document['Version'],
            'Date': document['Date'],
            'Confidentiality': document['Confidentiality'],
            'Perimeter': document['Perimeter'],
            'Investigation Number': document['Investigation Number'],
        }

        record_metadata = [{"chunk": j, "page_content": text, **metadata} for j, text in enumerate(processed_texts)]
        all_record_metadata.extend(record_metadata)

    return all_record_metadata

In [202]:
import os
from getpass import getpass
from semantic_router.encoders import OpenAIEncoder

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY") or getpass(
    "OpenAI API key: "
)

encoder = OpenAIEncoder(name="text-embedding-3-large")	

In [203]:
from semantic_chunkers import StatisticalChunker

chunker = StatisticalChunker(encoder=encoder)

In [220]:
def process_dataset(data):
    all_record_metadata = []

    for idx, document in data.iterrows():
        record_texts = chunker([document['Page Content']])
        processed_texts = []
        for chunk in record_texts[0]:
            text_chunk = chunk.splits
            text_chunk = " ".join(text_chunk)
            processed_texts.append(text_chunk)

        metadata = {
            'Title': document['Title'],
            'Version': document['Version'],
            'Date': document['Date'],
            'Confidentiality': document['Confidentiality'],
            'Perimeter': document['Perimeter'],
            'Investigation Number': document['Investigation Number'],
        }

        record_metadata = [{"chunk": j, "page_content": text, **metadata} for j, text in enumerate(processed_texts)]
        all_record_metadata.extend(record_metadata)

    return all_record_metadata

In [ ]:
import os
from semantic_router.encoders import OpenAIEncoder
from semantic_chunkers import StatisticalChunker
import pandas as pd

class DatasetChunker:
    def __init__(self):
        self.encoder = self.initialize_encoder()
        self.chunker = self.create_chunker(self.encoder)
    
    def initialize_encoder(self):
        # Set the OpenAI API key from environment variable
        os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
        # Initialize the encoder
        encoder = OpenAIEncoder(name="text-embedding-3-large")
        return encoder

    def create_chunker(self, encoder):
        return StatisticalChunker(encoder=encoder)

    def process_dataset(self, data):
        all_record_metadata = []

        for idx, document in data.iterrows():
            record_texts = self.chunker([document['Page Content']])
            processed_texts = []
            for chunk in record_texts[0]:
                text_chunk = chunk.splits
                text_chunk = " ".join(text_chunk)
                processed_texts.append(text_chunk)

            metadata = {
                'Title': document['Title'],
                'Version': document['Version'],
                'Date': document['Date'],
                'Confidentiality': document['Confidentiality'],
                'Perimeter': document['Perimeter'],
                'Investigation Number': document['Investigation Number'],
            }

            record_metadata = [{"chunk": j, "page_content": text, **metadata} for j, text in enumerate(processed_texts)]
            all_record_metadata.extend(record_metadata)

        return all_record_metadata


processor = DatasetChunker()
result = processor.process_dataset(data[:5])

result


In [8]:
data_test = process_dataset(data[:5])

2024-06-15 11:26:59 INFO semantic_chunkers.utils.logger Single document exceeds the maximum token limit of 300. Splitting to sentences before semantically merging.
100%|██████████| 4/4 [00:03<00:00,  1.12it/s]
2024-06-15 11:27:03 INFO semantic_chunkers.utils.logger Single document exceeds the maximum token limit of 300. Splitting to sentences before semantically merging.
100%|██████████| 4/4 [00:03<00:00,  1.29it/s]
2024-06-15 11:27:06 INFO semantic_chunkers.utils.logger Single document exceeds the maximum token limit of 300. Splitting to sentences before semantically merging.
100%|██████████| 4/4 [00:03<00:00,  1.14it/s]
2024-06-15 11:27:10 INFO semantic_chunkers.utils.logger Single document exceeds the maximum token limit of 300. Splitting to sentences before semantically merging.
100%|██████████| 4/4 [00:03<00:00,  1.11it/s]
2024-06-15 11:27:13 INFO semantic_chunkers.utils.logger Single document exceeds the maximum token limit of 300. Splitting to sentences before semantically mergi

In [ ]:
data_test

# Initizalize OpenAI Azure models

In [3]:
import os
from openai import AzureOpenAI
from dotenv import load_dotenv
from openai import OpenAI


def get_azure_openai_response(prompt):
    # Load environment variables
    load_dotenv()

    # Initialize the AzureOpenAI client
    completion_client = AzureOpenAI(
        azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
        api_key=os.getenv("AZURE_OPENAI_API_KEY"),
        api_version="2023-05-15",
        azure_deployment=os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME")
    )

    # Get response from the model
    response = completion_client.completions.create(
        model="gpt-35-turbo",
        prompt=prompt,
        temperature=0,
        max_tokens=3000,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0,
    )

    # Return the text of the first choice
    return response.choices[0].text

# Usage example
# prompt = "quelle est l'information sur le coronavirus ?"
# response_text = get_azure_openai_response(prompt)
# print(response_text)

In [4]:
from openai import OpenAI

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY") or getpass(
    "OpenAI API key: "
)

client_emb = OpenAI()

def get_embeddings(text):
   embeddings = client_emb.embeddings.create(input = """{}""".format(text), 
                                             model="text-embedding-ada-002",
                                             # length_function=tiktoken_len
                                             )
   return embeddings.data[0].embedding

In [ ]:
get_embeddings("quelle est l'information sur le coronavirus ?")

# Initizalize Blob Storage

In [6]:
from azure.storage.blob import BlobServiceClient
import os
from dotenv import load_dotenv
load_dotenv()

True

In [10]:
AZURE_STORAGE_CONNECTION_STRING:str=os.getenv("AZURE_STORAGE_CONNECTION_STRING")
if not AZURE_STORAGE_CONNECTION_STRING.startswith("DefaultEndpointsProtocol="):
    AZURE_STORAGE_CONNECTION_STRING = "DefaultEndpointsProtocol=" + AZURE_STORAGE_CONNECTION_STRING
AZURE_STORAGE_CONTAINER_NAME=os.getenv("AZURE_STORAGE_CONTAINER_NAME")
AZURE_STORAGE_API_KEY=os.getenv("AZURE_STORAGE_API_KEY")    
AZURE_STORAGE_ENDPOINT=os.getenv("AZURE_STORAGE_ENDPOINT")
AZURE_STORAGE_DEPLOYEMENT_ID=os.getenv("AZURE_STORAGE_DEPLOYEMENT_ID")
blob_service_client = BlobServiceClient.from_connection_string(AZURE_STORAGE_CONNECTION_STRING)


In [ ]:
import os
from azure.storage.blob import BlobServiceClient

def download_blob_to_file(blob_service_client: BlobServiceClient, container_name: str, blob_name: str):
    blob_client = blob_service_client.get_blob_client(container=container_name, blob=blob_name)
    file_name = blob_name.split("/")[-1]
    
    # Create a folder name with the -2 element of the blob_name if it does not exist in the downloaded_files folder
    subfolder = blob_name.split("/")[-2]
    os.makedirs(os.path.join(r'blob_files', subfolder), exist_ok=True)
    
    file_path = os.path.join(r'blob_files', subfolder, file_name)
    
    # Check if file already exists
    if os.path.exists(file_path):
        print(f"File already exists: {file_path}")
        return None
    
    with open(file_path, mode="wb") as sample_blob:
        download_stream = blob_client.download_blob()
        sample_blob.write(download_stream.readall())
    
    return file_path

def download_all_blobs_in_container(blob_service_client: BlobServiceClient, container_name: str):
    container_client = blob_service_client.get_container_client(container_name)
    blob_list = container_client.list_blobs()
    
    for blob in blob_list:
        downloaded_file_path = download_blob_to_file(blob_service_client, container_name, blob.name)
        if downloaded_file_path:
            print(f"Downloaded to: {downloaded_file_path}")
        else:
            print(f"Skipped: {blob.name}")
            
download_all_blobs_in_container(blob_service_client, AZURE_STORAGE_CONTAINER_NAME)

# initialize Search AI

In [2]:
from azure.search.documents.indexes import SearchIndexClient
from azure.search.documents.indexes.models import (
    # ComplexField,
    # CorsOptions,
    SearchIndex,   
    # ScoringProfile,
    SearchFieldDataType,
    SimpleField,
    SearchableField,
    SearchField,
    VectorSearch,
    VectorSearchProfile,
    HnswAlgorithmConfiguration
    )
from azure.search.documents import SearchClient
from azure.search.documents.indexes import SearchIndexClient
from azure.search.documents.models import VectorizedQuery
from azure.core.credentials import AzureKeyCredential
import os
import uuid
from tqdm import tqdm

In [3]:
AZURE_AI_SEARCH_API_KEY=os.getenv("AZURE_AI_SEARCH_API_KEY")
AZURE_AI_SEARCH_ENDPOINT=os.getenv("AZURE_AI_SEARCH_ENDPOINT")
AZURE_AI_SEARCH_DEPLOYEMENT_ID=os.getenv("AZURE_AI_SEARCH_DEPLOYEMENT_ID")
AZURE_AI_SEARCH_INDEX_NAME=os.getenv("AZURE_AI_SEARCH_INDEX_NAME")
AZURE_AI_SEARCH_INDEXER_NAME=os.getenv("AZURE_AI_SEARCH_INDEXER_NAME")
OPENAI_API_KEY=os.getenv("OPENAI_API_KEY")

In [4]:
vector_search = VectorSearch(
        profiles=[VectorSearchProfile(name="my-vector-config", algorithm_configuration_name="my-algorithms-config")],
        algorithms=[HnswAlgorithmConfiguration(name="my-algorithms-config",
                                               parameters={"distanceMeasure": "Cosine"})]
    )

In [5]:
# create a search index

index_client = SearchIndexClient(AZURE_AI_SEARCH_ENDPOINT, AzureKeyCredential(AZURE_AI_SEARCH_API_KEY))
index_name = AZURE_AI_SEARCH_INDEX_NAME
fields = [
    SimpleField(name="id", type=SearchFieldDataType.String, key=True),
    SearchableField(name="page_content", type=SearchFieldDataType.String, analyzer_name="fr.lucene"),
    SearchField(name="embeddings", 
                type=SearchFieldDataType.Collection(SearchFieldDataType.Single), 
                searchable = True,
                vector_search_dimensions=1536,
                vector_search_profile_name="my-vector-config"),
    SearchableField(name="Perimeter", type=SearchFieldDataType.String, sortable=True, filterable=True),
    SearchableField(name="Title", type=SearchFieldDataType.String, analyzer_name="fr.lucene"),
    SearchableField(name="Version", type=SearchFieldDataType.String, sortable=True, filterable=True),
    # SearchableField(name="Investigation Number", type=SearchFieldDataType.String, sortable=True, filterable=True),
    SearchableField(name="Date", type=SearchFieldDataType.String, sortable=True, filterable=True),
]

index = SearchIndex(name=index_name, fields=fields, vector_search=vector_search)
index_client.create_index(index)
# index_client.delete_index(index)

In [228]:
docs = []
for doc in data_test:
    docs.append({"id": str(uuid.uuid4()),
                 "page_content": doc['page_content'],
                 "embeddings": get_embeddings(doc['page_content']),
                 "Perimeter": doc['Perimeter'],
                 "Title": doc['Title'],
                 "Version": doc['Version'],
                 "Date": doc['Date'],
                 })

In [11]:
search_client = SearchClient(AZURE_AI_SEARCH_ENDPOINT,index_name=AZURE_AI_SEARCH_INDEX_NAME, credential = AzureKeyCredential(AZURE_AI_SEARCH_API_KEY))
result  = search_client.upload_documents(docs)

NameError: name 'SearchClient' is not defined

In [1]:
query = "quelle zones sont les zones plus investigue dans le perimetre DEG"

In [10]:
from azure.search.documents.models import VectorizedQuery

vector_query = VectorizedQuery(vector=get_embeddings(query),k_nearest_neighbors=20,  fields="embeddings")

results = search_client.search(
    vector_queries=[vector_query],
    select=["page_content"],
    query_language="fr-fr",)

NameError: name 'get_embeddings' is not defined

In [2]:
from VectorQuery import AzureSearchQuery

azure_client = AzureSearchQuery()

# Perform a vectorized query
input_text = azure_client.vectorized_query(query)
input_text

HttpResponseError: () Invalid expression: Could not find a property named 'source' on type 'search.document'.
Parameter name: $select
Code: 
Message: Invalid expression: Could not find a property named 'source' on type 'search.document'.
Parameter name: $select

In [6]:
input_text = ''
try:
    while True: 
        result = results.next()
        input_text += result["page_content"] + "\n "
except StopIteration:
    pass

NameError: name 'results' is not defined

In [248]:

prompt = f"""En utilisant le context suivant : {input_text}\n 
il faut repondre a la question suivante : {query} seulment en francais"""

In [ ]:
get_azure_openai_response(prompt)